## Qumulate Tutorial

In the following notebook we will upload data from a local excel sheet into Qumulate using Python. This tutorial can be used as a basis for uploading any Qumulate tests from a local data source. E.g. a database or file. As a TL;DR, the steps will be summarized here as a refresher if reading this for the second time:

1. Get the API key and API secret key
2. Get the test schema boilerplate from the Qumulate app for the test(s) to be uploaded
3. Change the values of the boilerplate to the real values
4. Create a security signature of the message (data) to be sent.
5. Actually send (POST) the message and signature to Qumulate. 

This notebook can be copied and used directly if the API_KEY, API_SECRET_KEY, and QLT_URL are appropriately set. 



#### Setup

First we import the libraries needed.

In [ ]:
!pip install requests

In [1]:
import base64
import hashlib
import hmac
import json

import requests

Second, we'll need the API keys so our request is authenticated. You can get these from the Qumulate app under Configuration -> API accounts. Select the account to use and copy the keys:

![title](qlt_api_key.png)

The URL to use will always be `https://qumulate.varian.com/public` for production. For this tutorial we're using the demo instance.

In [2]:
API_KEY = "copy your public key here"
API_SECRET_KEY = "copy your secret key here"
QLT_URL = 'https://demo.qumulate.varian.com/public'

### Getting the schema

We'll now need to grab the data schema that Qumulate is expecting to recieve. For each test, boilerplate is provided for easy copy and paste. To get the boilerplate for a given test, go to Configuration -> QUIP Documentation and select the test to get a file download of the schema. For our example we'll use Output consistency in JSON format. 

![title](qlt_json_schema.png)

Copy and pasting the file here shows the following:


In [3]:
{
  "machines": [
    {
      "serial-number": "H191234",
      "configuration": {
        "machine-configuration-values": [
          {
            "field-code": "ENERGY",
            "unit-code": "MEGA_ELECTRONVOLT",
            "value": "20"
          }
        ]
      },
      "tests": [
        {
          "device": {
            "type": "UNKNOWN",
            "serial-number": "123456789"
          },
          "performed-on-date": "24 Jul 2020 15:12:51 +0000",
          "data-values": [
            {
              "test-raw-data-value-code": "DOSE",
              "unit": "CENTIGRAY",
              "value": "1.5"
            }
          ],
          "custom-values": [
            {
              "key": "test-performed-by",
              "value": "John Doe"
            }
          ],
          "temperature": {
            "units": {
              "name": "CELSIUS"
            },
            "value": 21.0
          },
          "atmospheric-pressure": {
            "units": {
              "name": "MILLIMETERS_OF_MERCURY"
            },
            "value": 760.0
          }
        }
      ]
    }
  ]
}

{'machines': [{'serial-number': 'H191234',
   'configuration': {'machine-configuration-values': [{'field-code': 'ENERGY',
      'unit-code': 'MEGA_ELECTRONVOLT',
      'value': '20'}]},
   'tests': [{'device': {'type': 'UNKNOWN', 'serial-number': '123456789'},
     'performed-on-date': '24 Jul 2020 15:12:51 +0000',
     'data-values': [{'test-raw-data-value-code': 'DOSE',
       'unit': 'CENTIGRAY',
       'value': '1.5'}],
     'custom-values': [{'key': 'test-performed-by', 'value': 'John Doe'}],
     'temperature': {'units': {'name': 'CELSIUS'}, 'value': 21.0},
     'atmospheric-pressure': {'units': {'name': 'MILLIMETERS_OF_MERCURY'},
      'value': 760.0}}]}]}

The schema shows several key items including the machine serial number `H191234`, the `machine-configuration-values` that contains the energy `20` and modality `MEGA_ELECTRONVOLT`. In the `tests` section a `device` contains information if the data was acquired with a device such as the DailyQA3 or Doselab. The `performed-on-date` and `data-values` contain the critical information that will direct the information to the right Qumulate QA session and tests. 

### Creating a custom message

Let's edit this file to remove some superfluous keys (`temperature`, `atmospheric-pressure`) as well as wrap this with a function that can dynamically populate the `performed-on-date` and `value` of the `data-values` section. 

In [4]:
def create_json(perform_date, output_value, energy, serial_number):
    """
    Parameters
    ----------
    perform_date : string, date representation
    output_value : float, numerical representation of the dose in cGy.
    energy : int, energy in MV
    serial_number : string, full serial number of the linac
    """
    data = dict(
        {
            "machines": [
                {
                    "serial-number": str(serial_number),
                    "configuration": {
                        "machine-configuration-values": [
                            {
                                "field-code": "ENERGY",
                                "unit-code": "MEGA_ELECTRONVOLT",
                                "alt": "MV",
                                "value": str(energy)
                            }
                        ]
                    },
                    "tests": [
                        {
                            "device": {
                                "type": "UNKNOWN",
                                "serial-number": "H197771"
                              },
                            "performed-on-date": str(perform_date),
                            "data-values": [
                                {
                                    "test-raw-data-value-code": "DOSE",
                                    "unit": "CENTIGRAY",
                                    "value": str(output_value)
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    )
    data_as_str = json.dumps(data)
    return data_as_str

In the above code, we created a function that we can reuse to populate the important fields anytime we want to create a JSON message for Qumulate. Let's go ahead and create a message:

In [5]:
perform_date = "24 Jul 2020 15:12:53 +0000"
output_value = 100.5
energy = 6
serial_number = "H197771"
data = create_json(perform_date, output_value, energy, serial_number)
data

'{"machines": [{"serial-number": "H197771", "configuration": {"machine-configuration-values": [{"field-code": "ENERGY", "unit-code": "MEGA_ELECTRONVOLT", "alt": "MV", "value": "6"}]}, "tests": [{"device": {"type": "UNKNOWN", "serial-number": "H197771"}, "performed-on-date": "24 Jul 2020 15:12:53 +0000", "data-values": [{"test-raw-data-value-code": "DOSE", "unit": "CENTIGRAY", "value": "100.5"}]}]}]}'

### Formatting the message

We have the data we want to send to Qumulate, however, we must create a data "signature" so that the data is ensured to be secure during transit from our local script to the Qumulate cloud. Specifically, we use hashed message authentication code ([HMAC](https://en.wikipedia.org/wiki/HMAC)) with the [SHA256](https://decryptionary.com/dictionary/secure-hash-algorithm-256/) hashing algorithm. The hashing algorithm takes our API secret key as in input parameter. 

In [6]:
hmac_msg = hmac.new(key=API_SECRET_KEY.encode('utf-8'), msg=data.encode('utf-8'), digestmod=hashlib.sha256)
hmac_msg.hexdigest()

'26f3ae40100141402331b148c3e679cb53e1144a81242b0a6805d7d5c7f05fd1'

We now need to convert this HMAC message to [base64](https://en.wikipedia.org/wiki/Base64) format, which is the format Qumulate will accept:

In [7]:
b64_hash = base64.b64encode(hmac_msg.digest())
b64_hash

b'JvOuQBABQUAjMbFIw+Z5y1PhFEqBJCsKaAXX1cfwX9E='

### POST the message to Qumulate

We're now ready to send this message to Qumulate. We format an HTTP request which needs to know the URL destination, the payload (the message) and the API Key:

In [8]:
resp = requests.post(url=QLT_URL, data=data,
                     headers={'content-type': 'application/json',
                              'api_key': API_KEY,
                              'api_client_id': 'SI_V1',
                              'data_signature': b64_hash.decode('utf-8')})

In the above code we pushed our request via a POST call. We will get a response `resp` back which will give us the status of our request. Our request could be successful or fail. This response is the message Qumulate sends back after receiving the message. Let's see what our response says. We print it in JSON format for easier reading:

In [9]:
resp.json()

{'success': False,
 'request-validation-status': 'FAILED_BEFORE_VALIDATION',
 'api-response': 'API_LOOKUP_FAILED',
 'stats': {'processed-tests': 0,
  'new-tests': 0,
  'duplicate-tests': 0,
  'valid-tests': 0,
  'received-files': 0,
  'validated-files': 0},
 'status-message': 'API Lookup Failed : Maybe the API key was incorrect?'}

Our message failed to upload. This is because the API key was left at the default `API_KEY = "copy your public key here"` from the beginning of this tutorial. If we plug in a valid value for the key and secret key the response will look more like the following:

```
{'success': True,
 'request-validation-status': 'VALIDATED',
 'api-response': 'SUCCESS_AWAITING_MAPPING',
 'stats': {'processed-tests': 0,
  'new-tests': 0,
  'duplicate-tests': 0,
  'valid-tests': 0,
  'received-files': 0,
  'validated-files': 0},
 'status-message': 'Waiting for user to map the alternate-mapping values to their machine.'}
 ```
 
 This message indicates that we successfully sent the message. However, Qumulate was not able to match the serial number in the message (which was a hypothetical `H197771`) with a machine in the Qumulate organization. I.e. there was no `H197771` machine in the clinic's Qumulate machine list. N.B. Qumulate will not match machines outside the clinic's organization, meaning you can't accidentally send or receive data from another institution, even if the serial number was set to the same value. This is done through the API key, which only looks at the machines associated with the API key. 
 
Even though the serial number didn't match, Qumulate provides an ability to match serial numbers to real machines, so we're still okay. 


### That's it from the local side

At this point we've finished sending the message. We can use the schema from other tests to send other tests, but the idea is the same. Using the above example, we still have some cleanup to do in Qumulate to get the sent data to the right machine but is handled from the Qumulate app. For ease of use, a function is given below that generates the signature and sends to Qumulate. This can be copy and pasted:

In [10]:
def send_message(data, API_KEY, API_SECRET_KEY, QLT_URL):
    hmac_hash = hmac.new(key=API_SECRET_KEY.encode('utf-8'), msg=data.encode('utf-8'), digestmod=hashlib.sha256)
    b64_hash = base64.b64encode(hmac_hash.digest())
    resp = requests.post(url=QLT_URL, data=data,
                         headers={'content-type': 'application/json',
                                  'api_key': API_KEY,
                                  'api_client_id': 'SI_V1',
                                  'data_signature': b64_hash.decode('utf-8')})
    return json.dumps(resp.json())

### Match the machine in Qumulate

This step is not needed if the serial number of the message actually matched a machine defined for the clinic in Qumulate. For this example we'll show how to do it. 

Going to Qumulate we see the message sent above by going to Configuration -> API Accounts and viewing the API Request History:

![title](qlt_request_history.png)

We can see our request is in triage, waiting to match to a real Qumulate machine. To do so, go to the next tab, Machine Mapping:

![title](qlt_machine_mapping.png)

Click on `Link` to match the number to a real machine:

![title](qlt_machine_mapping_selection.png)

After matching, Qumulate will ask you if you want to process the requests that are waiting, i.e. the request we just sent. We want to do that so we'll say Yes. If you choose to say No you can resubmit the requests later:

![title](qlt_resubmit.png)

Once resubmitted, the values will then get sent to the template that contains the Output Consistency test. We can see the test result in the QA session for the matched machine and assignment:

![title](qlt_session_data.png)

There it is! A test data point sent from start to finish. The process of matching need only be done once. After that data will automatically map.

## Excel upload example

The above example can be extended to send Excel data up to Qumulate. The below example will grab cells from a local excel sheet and use those values as input parameters to the functions defined above. The only part that's new is grabbing the cells from the sheet. After that, all else is the same.

### Getting the data from Excel

First, we'll need to import a library that reads Excel files:

In [11]:
!pip install openpyxl

In [12]:
from openpyxl import load_workbook

In [13]:
wb = load_workbook('output-example.xlsx', data_only=True)  # this opens the file. data_only means grab the values from formula cells, not the forumlas themselves
ws = wb.active  # this grabs the "active" worksheet. This file only has one, but you can select a different one if your file has multiple sheets

We now have a reference to the workbook (`wb`) and active worksheet (`ws`). We can now grab individual cells:

In [14]:
ws['H9'].value

'H197771'

So then let's grab the three values from the spreadsheet we want and then use them in our functions defined above:

In [15]:
serial_number = ws['H9'].value
energy = ws['H10'].value
output_value = ws['H15'].value

### Date handling

We didn't grab a time from our Excel sheet, but we can if we want to. For this example we will simply use "now" as the perform time/date. One important caveat is that Qumulate is time-zone aware. Thus, we also must be time-zone aware when uploading data. E.g. uploading data at "06:00" doesn't mean much because Qumulate doesn't know which time zone you're in. Below we grab the current time and convert it to a time-zone aware format:

In [20]:
!pip install pytz

In [16]:
from datetime import datetime
from pytz import timezone

central = timezone("US/Central")  # pick whatever zone you're in

now = datetime.now()  

loc_now = central.localize(now)  # wrap the current time with the timezone

 # current date and time
print("The current time without timezone awareness:", now)
print("The current time with timezone awareness:", loc_now)

The current time without timezone awareness: 2020-07-24 12:58:08.631819
The current time with timezone awareness: 2020-07-24 12:58:08.631819-05:00


Note the `-0500` at the end of the time-zone aware datetime. 

Now, we must convert the timestamp to a string in the format Qumulate expects. See [here](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) for more info on formatting.



In [17]:
fmt = '%d %b %Y %H:%M:%S %z'
localized_time_as_str = loc_now.strftime(fmt)
print(localized_time_as_str)

24 Jul 2020 12:58:08 -0500


This string is now in the format Qumulate expects. Note the timezone shift at the end. 

### Uploading data

Now that we have the data from Excel and a proper datetime stamp we are ready to upload:

In [18]:
# copy of the above data for clarity
serial_number = ws['H9'].value
energy = ws['H10'].value
output_value = ws['H15'].value
# set the date to the stamp we 
perform_date = localized_time_as_str

data = create_json(perform_date, output_value, energy, serial_number)
data

'{"machines": [{"serial-number": "H197771", "configuration": {"machine-configuration-values": [{"field-code": "ENERGY", "unit-code": "MEGA_ELECTRONVOLT", "alt": "MV", "value": "6"}]}, "tests": [{"device": {"type": "UNKNOWN", "serial-number": "H197771"}, "performed-on-date": "24 Jul 2020 12:58:08 -0500", "data-values": [{"test-raw-data-value-code": "DOSE", "unit": "CENTIGRAY", "value": "100.75315249999998"}]}]}]}'

In [19]:
r = send_message(data, API_KEY, API_SECRET_KEY, QLT_URL)
r

'{"success": false, "request-validation-status": "FAILED_BEFORE_VALIDATION", "api-response": "API_LOOKUP_FAILED", "stats": {"processed-tests": 0, "new-tests": 0, "duplicate-tests": 0, "valid-tests": 0, "received-files": 0, "validated-files": 0}, "status-message": "API Lookup Failed : Maybe the API key was incorrect?"}'

Although this request failed (we still have dummy key values, remember?) the response is the same as our original example above. With valid key values the request will work. 